In [1]:
import sys
import requests
from bs4 import BeautifulSoup

def crawl_products(cg_link, cg_id):

    product_json = []
    page = 1
    
    while True:
        
        try:
            url = f'{cg_link}&page={page}'
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
            responses = requests.get(headers=headers, url=url, timeout=100)
            responses.raise_for_status()
        except requests.exceptions.RequestException as e:
            continue

        soup = BeautifulSoup(responses.content, 'html.parser')

        products_block = soup.find_all('div', class_='product-block desktop-pdt')

        if len(products_block) > 0:

            for product in products_block:

                data = {}
                
                p_image = product.find('img', class_='img-responsive lazy-load').attrs['src']

                p_name = product.find('div', class_='name').text

                p_price = int(product.find('span', class_='price-new').text.replace('₫','').replace('.',''))

                p_link = product.find('div', class_='name').find('a').attrs['href']

                p_id = p_link.split('&')[-1].replace('product_id=', '')

                cg_id = cg_id

                data['p_image'] = p_image
                data['p_id'] = p_id
                data['p_name'] = p_name.strip()
                data['p_price'] = p_price
                data['cg_id'] = cg_id
                data['p_link'] = p_link

                product_json.append(data)
            
            sys.stdout.write(f'Category ID: {cg_id} | Page: {page} | Total Products: {len(product_json)}\r')
            sys.stdout.flush()
            
        else:
            print(f'{len(product_json)} Products of Category {cg_id} have been added successfully')
            break

        page += 1
    
    return product_json

def crawl_categories():

    try:
        responses = requests.get('https://sala.emartmall.com.vn/index.php?route=common/home', timeout=100)
        responses.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f'Error fetching in Homepage: {e}')
    
    soup = BeautifulSoup(responses.content, 'html.parser')
    
    categories = soup.find_all('li', class_='parent dropdown')

    categories_json = []

    products_json = []
    
    for category in categories:

        data = {}
        
        cg_name = category.find('span', class_='menu-title').text

        cg_link = category.find('a').attrs['href']

        cg_id = cg_link.split('&')[-1].replace('path=', '')
        
        data['cg_id'] = cg_id
        data['cg_name'] = cg_name.strip()
        data['cg_link'] = cg_link

        categories_json.append(data)

        product_json = crawl_products(cg_link, cg_id)

        products_json += product_json

    return categories_json, products_json
    

In [2]:
categories_json, products_json = crawl_categories()

Error fetching in Homepage: HTTPSConnectionPool(host='sala.emartmall.com.vn', port=443): Max retries exceeded with url: /index.php?route=common/home (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F9805E0CD0>, 'Connection to sala.emartmall.com.vn timed out. (connect timeout=100)'))


UnboundLocalError: cannot access local variable 'responses' where it is not associated with a value

In [4]:
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

responses = requests.get(headers=headers, url='https://sala.emartmall.com.vn/index.php?route=common/home')
    
print(responses)

ConnectTimeout: HTTPSConnectionPool(host='sala.emartmall.com.vn', port=443): Max retries exceeded with url: /index.php?route=common/home (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000023B461A2490>, 'Connection to sala.emartmall.com.vn timed out. (connect timeout=None)'))

In [19]:
from pandas import json_normalize

df_categories = json_normalize(categories_json)
df_products = json_normalize(products_json)

In [242]:
import pyodbc

# Parameter for connection
server = 'DESKTOP-AL8SAM5'
database = 'EmartShopping'
username = 'sa'
password = 'huan181102'

# Connection SQL Server
try:
    conn = pyodbc.connect('DRIVER={SQL Server};' + f'SERVER={server};DATABASE={database};UID={username};PWD={password}')
    cursor = conn.cursor()
    print('Connection SQL Server Successfully')
except:
    print('Connection SQL Server Unsuccessfully')

# Insert Categories
try:
    for index, row in df_categories.iterrows():
        cursor.execute("INSERT INTO Categories (cg_id, cg_name, cg_link) VALUES(?,?,?)", row.cg_id, row.cg_name, row.cg_link)
        sys.stdout.write(f'Index {index} of Categories has been inserted to the SQL Server\r')
        sys.stdout.flush()
except requests.exceptions.RequestException as e:
    print(f'Error at Index {index} of Categories: {e}')

# Insert Products
try:
    for index, row in df_products.iterrows():
        cursor.execute("INSERT INTO Products (p_image, p_id, p_name, p_price, cg_id, p_link) VALUES(?,?,?,?,?,?)", row.p_image, row.p_id, row.p_name, row.p_price, row.cg_id, row.p_link)
        sys.stdout.write(f'Index {index} of Products has been inserted to the SQL Server\r')
        sys.stdout.flush()
except requests.exceptions.RequestException as e:
    print(f'Error at Index {index} of Products: {e}')

print('All data have been inserted successfully')

cursor.commit()
cursor.close()

Connection Successfully
Insert Completed


In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head()

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [2]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Kidney Beans, Eggs)"
6,0.6,"(Eggs, Onion)"
7,0.6,"(Kidney Beans, Milk)"
8,0.6,"(Kidney Beans, Onion)"
9,0.6,"(Kidney Beans, Yogurt)"


In [3]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.0,0.0
1,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf,0.0
2,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0
3,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
4,(Milk),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
5,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
6,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0
7,"(Kidney Beans, Eggs)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,1.0
8,"(Kidney Beans, Onion)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,0.5
9,"(Eggs, Onion)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf,0.0


In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

data = {
    'TransactionID': [1, 1, 1, 2, 2, 3, 3, 3],
    'Item': ['Bread', 'Butter', 'Milk', 'Butter', 'Milk', 'Bread', 'Milk', 'Butter']
}
df = pd.DataFrame(data)
print(df)
grouped = df.groupby('TransactionID')['Item'].apply(list)
print(grouped)

list_grouped = list(grouped)

te = TransactionEncoder()
te_ary = te.fit(list_grouped).transform(list_grouped)
df_new = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df_new, min_support=0.6, use_colnames=True)
print(frequent_itemsets)

df_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.000001)

df_rules

   TransactionID    Item
0              1   Bread
1              1  Butter
2              1    Milk
3              2  Butter
4              2    Milk
5              3   Bread
6              3    Milk
7              3  Butter
TransactionID
1    [Bread, Butter, Milk]
2           [Butter, Milk]
3    [Bread, Milk, Butter]
Name: Item, dtype: object
    support               itemsets
0  0.666667                (Bread)
1  1.000000               (Butter)
2  1.000000                 (Milk)
3  0.666667        (Butter, Bread)
4  0.666667          (Milk, Bread)
5  1.000000         (Butter, Milk)
6  0.666667  (Milk, Butter, Bread)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Butter),(Bread),1.000000,0.666667,0.666667,0.666667,1.0,0.0,1.0,0.0
1,(Bread),(Butter),0.666667,1.000000,0.666667,1.000000,1.0,0.0,inf,0.0
2,(Milk),(Bread),1.000000,0.666667,0.666667,0.666667,1.0,0.0,1.0,0.0
3,(Bread),(Milk),0.666667,1.000000,0.666667,1.000000,1.0,0.0,inf,0.0
4,(Butter),(Milk),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf,0.0
5,(Milk),(Butter),1.000000,1.000000,1.000000,1.000000,1.0,0.0,inf,0.0
6,"(Butter, Milk)",(Bread),1.000000,0.666667,0.666667,0.666667,1.0,0.0,1.0,0.0
7,"(Bread, Milk)",(Butter),0.666667,1.000000,0.666667,1.000000,1.0,0.0,inf,0.0
8,"(Butter, Bread)",(Milk),0.666667,1.000000,0.666667,1.000000,1.0,0.0,inf,0.0
9,(Milk),"(Butter, Bread)",1.000000,0.666667,0.666667,0.666667,1.0,0.0,1.0,0.0
